In [1]:
import numpy as np
import pandas as pd

In [ ]:
# Read csv files
assemblyai = pd.read_csv('Diarization results/assemblyai_diarization_der_0.1272_30.csv')
deepgram = pd.read_csv('Diarization results/deepgram_diarization_der_0.1421_30.csv')
nemo = pd.read_csv('Diarization results/nemo_diarization.csv')
pyannote = pd.read_csv('Diarization results/pyannote_diarization_der_0.2130_30.csv')
reverb = pd.read_csv('Diarization results/reverb_diarization_der_0.2687_30.csv')
soniox = pd.read_csv('Diarization results/soniox_diarization_der_0.2005_30.csv')
